In [5]:
import torch
import polars as pl
import utils.model as model_utils
from tqdm.auto import tqdm  # Add this at top if not already
FILE_PATH = "D:/finetune_sbert_new/1Membeddings/result_embeddings/"

FILE_PATH_SAVE = "D:/finetune_sbert_new/1Membeddings/lora_16_quantized/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = FILE_PATH_SAVE + "quantized_lora_16_1000_5.pt"

# 1. load model

In [6]:
# load model 
model = model_utils.initialize_model()
model.load_state_dict(torch.load(model_name)['model_state_dict'])

<All keys matched successfully>

In [ ]:
embedding_name = "sapbert_lora_triplet16"
model.eval()
model.to(device)


# 2. embed quantized data

In [ ]:
# Load the dataset
embeddings = torch.load(FILE_PATH + embedding_name +".pt")

embedding_exp = embeddings["expressions_embeddings"].to(model_utils.Config.device)
embedding_label = embeddings["labels_embeddings"].to(model_utils.Config.device)
embedding_exp_q,_ = model_utils.evaluate(model, embedding_exp)
embedding_label_q,_ = model_utils.evaluate(model, embedding_label)


Average cosine similarity: 0.8048
Average cosine similarity: 0.7939


In [12]:

torch.save({
    "embedding_exp_q": embedding_exp_q.cpu(),   # move to CPU for saving
    "embedding_label_q": embedding_label_q.cpu()
}, FILE_PATH_SAVE + embedding_name + "_quantized.pt")

# embed synonyme quantized

In [25]:
syn_embeddings = torch.load(FILE_PATH + embedding_name +"_fd_syn_.pt")

embedding_syn = syn_embeddings["synonyme"]
embedding_exp = syn_embeddings["expressions_embeddings"].to(model_utils.Config.device)
idx = syn_embeddings['idx']
# embedding_syn_q,_ = model_utils.evaluate(model, embedding_syn)
embedding_exp_q,_ = model_utils.evaluate(model, embedding_exp)

Average cosine similarity: 0.7729


In [ ]:

torch.save(
    {
        "synonyme": embedding_syn,  # move to CPU for saving
        "expressions_embeddings": embedding_exp_q.cpu(),
        "idx": idx,
    },
    FILE_PATH_SAVE + embedding_name + "_syn_quantized.pt",
)


# is_a_invariant

In [27]:
embedding_new_expressions = torch.load(FILE_PATH + embedding_name +"new_exp.pt")

embedding_new_exp = embedding_new_expressions["new_expressions_embeddings"].to(model_utils.Config.device)
idx_true = embedding_new_expressions["idx_true"]
# embedding_syn_q,_ = model_utils.evaluate(model, embedding_syn)
embedding_exp_q,_ = model_utils.evaluate(model, embedding_new_exp)


torch.save(
    {
        "new_expressions_embeddings": embedding_exp_q.cpu(),  # move to CPU for saving
        "idx_true": idx_true,
    },
    FILE_PATH_SAVE + embedding_name + "_new_exp_quantized.pt",
)

Average cosine similarity: 0.7462
